In [1]:
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent))

from datetime import date, timedelta
import codecs
import pandas as pd
import numpy as np


pd.set_option('display.max_columns', None)

from os.path import join

from bulletin import default_input, default_output
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash, normalize_number
from bulletin.utils import static
from bulletin.utils.xls_ import fit_cols
from bulletin.systems.esus import eSUS

from datetime import datetime


today = datetime.today()
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)


---------------------------------------------------------------

root: C:\SESA\dev\bulletin
parent: C:\SESA\dev
default_input: C:\SESA\dev\input
default_output: C:\SESA\dev\output

---------------------------------------------------------------



In [2]:
esus = eSUS()
esus.load()

cols = ['cns','paciente','cpf','sexo','nome_mae','data_nascimento','idade','ibge_residencia','categoria','data_aplicacao','fabricante','grupo_atendimento','vacina_nome','dose','ibge_atendimento','hash_mae','hash_nasc']

In [4]:
if esus.df['cns'].isna().sum() != 0:
    raise Exception(f'{esus.df['cns'].isna().sum()} cns com valor nulo')

In [5]:
esus1dose = esus.df.loc[esus.df['dose']==1]
esus2dose = esus.df.loc[esus.df['dose']!=1]

In [6]:
vacinados = pd.merge(esus1dose,esus2dose,on='cns',how='outer',suffixes=['_1a_dose','_2a_dose'])

In [7]:
cols.remove('cns')
for col in cols:
    vacinados.loc[vacinados[f"{col}_1a_dose"].notna(), col] = vacinados.loc[vacinados[f"{col}_1a_dose"].notna(), f"{col}_1a_dose"]
    vacinados.loc[(vacinados[f"{col}_1a_dose"].isna()) & (vacinados[f"{col}_2a_dose"].notna()), col] = vacinados.loc[(vacinados[f"{col}_1a_dose"].isna()) & (vacinados[f"{col}_2a_dose"].notna()), f"{col}_2a_dose"]
    vacinados.loc[(vacinados[f"{col}_1a_dose"].isna()) & (vacinados[f"{col}_2a_dose"].isna()), col] = None

vacinados['idade'] = vacinados['idade'].fillna(-99).astype(int)
vacinados['ibge_residencia'] = vacinados['ibge_residencia'].fillna(-999999).astype(int)
vacinados['ibge_atendimento'] = vacinados['ibge_atendimento'].fillna(-999999).astype(int)

In [8]:
# vacinados.loc[(vacinados['paciente_2dose'].notna()) & (vacinados['paciente_1dose'].notna()) & (vacinados['paciente_1dose']!=vacinados['paciente_2dose']),['paciente_1dose','paciente_2dose']]

In [9]:
vacinados = vacinados[['cns','paciente','cpf','sexo','nome_mae','data_nascimento','idade','ibge_residencia','ibge_atendimento','categoria','grupo_atendimento','fabricante','vacina_nome','data_aplicacao_1a_dose','data_aplicacao_2a_dose','hash_mae','hash_nasc']]

In [10]:
notifica = Notifica()
notifica.load()
casos = notifica.df

In [11]:
casos.loc[casos['nome_mae'].notna(),'hash_mae'] = ( casos.loc[casos['nome_mae'].notna(),'paciente'].apply(normalize_hash) +
                                                          casos.loc[casos['nome_mae'].notna(),'nome_mae'].apply(normalize_hash) )

casos.loc[casos['data_nascimento'].notna(),'hash_nasc'] = ( casos.loc[casos['data_nascimento'].notna(),'paciente'].apply(normalize_hash) +
                                                                  casos.loc[casos['data_nascimento'].notna(),'data_nascimento'].apply(date_hash) )


In [12]:
casos_vacinados_cns = pd.merge(vacinados,casos[['cns','id','data_diagnostico','evolucao','data_cura_obito']], on='cns', how='inner')
print(len(casos_vacinados_cns))

casos_vacinados_cpf = pd.merge(vacinados.loc[vacinados['cpf'].notna()],casos[['cpf','id','data_diagnostico','evolucao','data_cura_obito']], on='cpf', how='inner')
print(len(casos_vacinados_cpf))

casos_vacinados_mae = pd.merge(vacinados.loc[vacinados['hash_mae'].notna()],casos[['hash_mae','id','data_diagnostico','evolucao','data_cura_obito']], on='hash_mae', how='inner')
print(len(casos_vacinados_mae))

casos_vacinados_nasc = pd.merge(vacinados.loc[vacinados['hash_nasc'].notna()],casos[['hash_nasc','id','data_diagnostico','evolucao','data_cura_obito']], on='hash_nasc', how='inner')
print(len(casos_vacinados_nasc))

casos_vacinados = pd.concat([casos_vacinados_cns,casos_vacinados_cpf,casos_vacinados_mae,casos_vacinados_nasc])
casos_vacinados = casos_vacinados.drop_duplicates('id', keep='first')
print(len(casos_vacinados))

267681
1050
257822
306512
291843


In [13]:
casos_vacinados[['id','evolucao']].groupby('evolucao').count()

,id
evolucao,
0.0,650
1.0,194311
2.0,3909
3.0,63392
4.0,44
